# 🚀 Servicios de Datos con FastAPI

Objetivo: construir un microservicio de datos con FastAPI usando modelos Pydantic, endpoints seguros y pruebas básicas, incorporando caché y límites de tasa opcionales.

- Duración: 90–120 min
- Dificultad: Media
- Prerrequisitos: Python intermedio, HTTP básico, Pydantic

## 0. Dependencias

- Instalar para ejecutar: `fastapi`, `uvicorn`, `pydantic` (si aún no disponible).
- Opcional: `redis` para caché externa y `slowapi` para rate limiting.
- Este notebook muestra el código; para correr el servidor, usa Uvicorn fuera del notebook.

## 1. App básica de FastAPI

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field
from datetime import datetime

# Código en un módulo src/app.py (ilustrativo dentro del notebook)
app_code = r'''
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, Field
from typing import List
from functools import lru_cache

app = FastAPI(title='Servicio de Datos Demo')

class Venta(BaseModel):
    venta_id: int = Field(..., gt=0)
    cliente_id: int
    total: float = Field(..., ge=0)

_DB = {1: Venta(venta_id=1, cliente_id=10, total=100.0)}

@lru_cache(maxsize=1024)
def get_precio_producto(producto_id: int) -> float:
    return 42.0

@app.get('/ventas', response_model=List[Venta])
def listar_ventas():
    return list(_DB.values())

@app.get('/ventas/{venta_id}', response_model=Venta)
def obtener_venta(venta_id: int):
    v = _DB.get(venta_id)
    if not v:
        raise HTTPException(404, 'No existe')
    return v

@app.post('/ventas', response_model=Venta, status_code=201)
def crear_venta(v: Venta):
    if v.venta_id in _DB:
        raise HTTPException(409, 'Duplicado')
    _DB[v.venta_id] = v
    return v

'''
print(app_code.splitlines()[:30])

### 1.1 Ejecutar con Uvicorn (opcional)

In [ ]:
print('Para ejecutar en terminal:')
print('uvicorn src.app:app --reload --port 8000')

## 2. Pruebas con requests (smoke test)

In [ ]:
import textwrap
smoke_test = textwrap.dedent('''
# Ejecutar con el servidor levantado
import requests
base = 'http://localhost:8000'
print(requests.get(f'{base}/ventas').json())
''')
print(smoke_test)

## 3. Extensiones: caché Redis y rate limit [opcional]

- Sustituir @lru_cache por Redis para cache distribuida.
- Usar slowapi (Flask-Limiter para FastAPI) para límites por IP/endpoint.
- Añadir autenticación JWT cuando expongas datos sensibles.